<a href="https://colab.research.google.com/github/NahuelCostaCortez/AI_notebooks/blob/main/OpenAI_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🗣️ **Whisper**

[Whisper](https://github.com/openai/whisper) is a general-purpose speech recognition model. It is trained on a large dataset of diverse audio and is also a multi-task model that can perform multilingual speech recognition as well as speech translation and language identification.

🚨 Make sure you are using a GPU. Go to `"Runtime" > "Change runtime type"` and in `"Hardware accelerator"` select `"GPU"`

## Install necessary libraries

In [ ]:
#@title ⚙️ Execute this cell to install the necessary libraries.
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-8svp21xn
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-8svp21xn
     |████████████████████████████████| 5.5 MB 16.8 MB/s 
     |████████████████████████████████| 7.6 MB 44.6 MB/s 
     |████████████████████████████████| 182 kB 65.2 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175239 sha256=e5310d9fd2852e3111ed5c11159a260f7d0c967e551a1e96a541f00ef740b295
  Stored in directory: /tmp/pip-ephem-wheel-cache-wpg9k1my/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 34.4 MB/s 
     |████████████████████████████████| 2.2 MB 50.9 MB/s 


## 🎙️ **Record and transcribe** [Choose the task]

In [ ]:
Task = "Translate to English" #@param ["Transcript to Language", "Translate to English"]
import numpy as np
import whisper
from scipy.io.wavfile import write
from IPython.display import clear_output

sel_task = "translate" if Task == "Translate to English" else "transcribe" 

audio, sr = get_audio()
write('record.wav', sr, audio)

!whisper "record.wav" --task {sel_task} --model medium --verbose False

clear_output()
if sel_task == "translate":
  print("-- TRADUCCIÓN A INGLÉS --\n")
else:
  print("-- TRANSCRIPCIÓN A ESPAÑOL --\n")
  
print(open('record.wav.txt').read())

## 👉  Only **transcribe** audio to the original language. (X ► X)

Just upload the audio you want to work with to the "Files" tab on the left. Then, run the cell below to transcribe the audio.

In [ ]:
!whisper "/content/audio.mp3" --task transcribe --model medium --verbose False --output_dir audio_transcription

tcmalloc: large alloc 1528012800 bytes == 0xa0b6000 @  0x7fc6c24061e7 0x4b2590 0x5ad01c 0x5dcfef 0x58f92b 0x590c33 0x5e48ac 0x4d20fa 0x51041f 0x58fd37 0x50c4fc 0x5b4ee6 0x58ff2e 0x50d482 0x58fd37 0x50c4fc 0x5b4ee6 0x6005a3 0x607796 0x60785c 0x60a436 0x64db82 0x64dd2e 0x7fc6c2003c87 0x5b636a
Detected language: Spanish
100% 338796/338796 [13:50<00:00, 407.85frames/s]


It can also be used in code, instead of from the terminal:



```
import whisper

model = whisper.load_model("medium")
result = model.transcribe("audio.mp3")
print(result["text"])
```



## 👉  Only **translate** audio from the original language to English. (X ► English)

Just upload the audio you want to work with to the "Files" tab on the left. Then, run the cell below to transcribe the audio.

In [ ]:
!whisper "/content/audio.mp3" --task translate --model medium